In [2]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats
import pandas as pd
import datetime as dt
import numpy_financial as npf
import yfinance as yf
from scipy.optimize import minimize

plt.style.use("ggplot")

# <font face="gotham" color="purple"> Capital Annual Growth Rate </font>

$$
C A G R=\left[\left(\frac{E V}{B V}\right)^{1/n}-1\right]\times 100
$$
where:
$E V=$ Ending value<br>
$B V=$ Beginning value<br>
$n=$ Number of years

In [5]:
sp500 = yf.download(
    ["^GSPC"],
    start=dt.datetime.today() - dt.timedelta(days=1500),
    end=dt.datetime.today(),
    progress=True,
    actions="inline",
    interval="1d",
)

[*********************100%***********************]  1 of 1 completed


This function is for daily data, because we use ```252``` number of trading days in a year to calculate $n$.

In [27]:
def get_cagr(df):
    EV = df["Close"][-1]
    BV = df["Close"][0]
    n = len(df) / 252
    cagr = (EV / BV) ** (1 / n) - 1
    print("CAGR: {:.2f}%".format(cagr * 100))
    return cagr

In [28]:
cagr = get_cagr(sp500)

CAGR: 9.64%


# <font face="gotham" color="purple"> Volatility </font>

Simple as you have imagined, volatility is commonly measured by standard deviation. The only thing you need to take heed is to know how to convert to annualized volatility. 
$$
\text{daily return} \times \sqrt{252}\\
\text{weekly return} \times \sqrt{52}\\
\text{monthly return} \times \sqrt{12}
$$

In [32]:
def get_volatility(df, freq):
    """
    The function is for using daily trading data.
    """
    daily_ret = df["Close"].pct_change().std()
    if freq == "daily":
        vol = daily_ret * np.sqrt(252)
    elif freq == "weekly":
        vol = daily_ret * np.sqrt(52)
    elif freq == "monthly":
        vol = daily_ret * np.sqrt(12)
    return vol

In [33]:
get_volatility(sp500, "daily")

0.23048979214458895

# <font face="gotham" color="purple"> Sharpe Ratio and Sortino Ratio </font>

$$
Sh.R = \frac{r_p - r_{f}}{\sigma_p}
$$

In [102]:
cisco = yf.download(
    ["CSCO"],
    start=dt.datetime.today() - dt.timedelta(days=3500),
    end=dt.datetime.today(),
    progress=True,
    actions="inline",
    interval="1d",
)

[*********************100%***********************]  1 of 1 completed


In [103]:
def get_sharpe(df, rf):
    sr = (get_cagr(df) - rf) / get_volatility(df, "daily")
    print("Risk-free rate:{}%".format(rf * 100))
    print("Sharpe Ratio: {}".format(sr))
    return sr

In [104]:
sr = get_sharpe(cisco, 0.045)

CAGR: 7.24%
Risk-free rate:4.5%
Sharpe Ratio: 0.10847264982490404


$$
So. R = \frac{r_p-r_f}{\sigma_p^*}
$$

where $\sigma_p^*$ only takes account of negative volatility.

In [105]:
def get_sortino(df, rf):
    df["daily_ret"] = df["Close"].pct_change()
    neg_volatility = df["daily_ret"][df["daily_ret"] < 0].std() * np.sqrt(252)
    sor = (get_cagr(df) - rf) / neg_volatility
    print("Risk-free rate:{}%".format(rf * 100))
    print("Sortino Ratio: {}".format(sor))
    return sor


sor = get_sortino(cisco, 0.045)

CAGR: 7.24%
Risk-free rate:4.5%
Sortino Ratio: 0.13696458182646315


# <font face="gotham" color="purple"> Max Drawdown and Calmar Ratio </font>

Max drawdown is the percentage counts the maximum drop from peak return in a certain period.

In [123]:
def get_max_dd(df):
    df["daily_ret"] = df["Close"].pct_change()
    df["cum_ret"] = (1 + df["daily_ret"]).cumprod()
    df["cum_trailing_max"] = df["cum_ret"].cummax()
    df["drawdown"] = df["cum_trailing_max"] - df["cum_ret"]
    df["drawdown_pct"] = df["drawdown"] / df["cum_trailing_max"]
    max_dd = df["drawdown_pct"].max()
    print("Max drawdown: {:.4f}%".format(max_dd * 100))
    return max_dd

In [124]:
mdd = get_max_dd(cisco)

Max drawdown: 42.8079%


Calmar ratio is similar to Sharpe ratio, but the risk is replace by maximum drawdown. A hedge fund manager named Terry W. Young invented Calmar ratio, and 'Calmar' is an acronym of his company's name and its newsletter: **CAL**ifornia **M**anaged **A**ccounts **R**eports.

In [125]:
def get_calmar(df):
    calmar = get_cagr(df) / get_max_dd(df)
    print("Calmar ratio: {:.4f}%".format(calmar))
    return calmar

In [126]:
get_calmar(cisco)

CAGR: 7.24%
Max drawdown: 42.8079%
Calmar ratio: 0.1692%


0.16921135956307007